<a href="https://colab.research.google.com/github/Ayman-Mansour/SDN-Dialect-Automatic-Speech-Recognition/blob/main/SDN_Dialect_XLSR_Wav2Vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fine-tuning XLSR-Wav2Vec2 on Sudanese Dialect** #

This a follow up to the first Sudanese dialect ASR model using CNN and CTC based on DeepSpeech design [A Proposed Automatic Speech Recognition model for the Sudanese Dialect 	](http://repository.sustech.edu/handle/123456789/25521?show=full), in this notebook we investigate the use of Wav2Vec2 pretrained model for ASR [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.
As Wav2Vec2 showed powerful performance on LibriSpeech Englis ASR dataset (click [here](https://arxiv.org/abs/2006.13979)).
XLSR *cross-lingual  speech representations* XLSR-Wav2Vec2 is able to learn speech representations that are going to be useful across multiple languages in this case the Sudanese Dialect.


Wav2Vec2 is a pretrained model for Automatic Speech Recognition (ASR) and was released in [September 2020](https://ai.facebook.com/blog/wav2vec-20-learning-the-structure-of-speech-from-raw-audio/) by Alexei Baevski, Michael Auli, and Alex Conneau.  Soon after the superior performance of Wav2Vec2 was demonstrated on the English ASR dataset LibriSpeech, *Facebook AI* presented XLSR-Wav2Vec2 (click [here](https://arxiv.org/abs/2006.13979)). XLSR stands for *cross-lingual  speech representations* and refers to XLSR-Wav2Vec2's ability to learn speech representations that are useful across multiple languages.

Similar to Wav2Vec2, XLSR-Wav2Vec2 learns powerful speech representations from hundreds of thousands of hours of speech in more than 50 languages of unlabeled speech. Similar, to [BERT's masked language modeling](http://jalammar.github.io/illustrated-bert/), the model learns contextualized speech representations by randomly masking feature vectors before passing them to a transformer network.

![wav2vec2_structure](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/xlsr_wav2vec2.png)

The authors show for the first time that massively pretraining an ASR model on cross-lingual unlabeled speech data, followed by language-specific fine-tuning on very little labeled data achieves state-of-the-art results. See Table 1-5 of the official [paper](https://arxiv.org/pdf/2006.13979.pdf).

As showwn in the previous reserch the Sudanese dialect concidered a low-resourced language, because of its dataset merky over 3 hours of labeled speech, hence by using XLSR-Wav2Vec2 pretrained model we are going to take advantage of speech representations.
Both researches share the use of Connectionist Tempoaral Classification (CTC) which is used to train neural networks  for sequence-to-sequence problems and mainly in Automatic Speech Recognition and handwriting recognition.

##Installation##
Before we start, let's install both `datasets` and `transformers` from master. Also, we need the `torchaudio` and `librosa` package to load audio files and the `jiwer` to evaluate our fine-tuned model using the [word error rate (WER)](https://huggingface.co/metrics/wer) metric ${}^1$.

In [ ]:
%%capture
!pip install datasets
!pip install transformers
!pip install torchaudio
!pip install librosa
!pip install jiwer
!pip install lang-trans

##Loading SDN dialect dataset##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/SDN Dialect Corpus v1.0.zip" "/content/"

In [ ]:
!unzip SDN\ Dialect\ Corpus\ v1.0.zip

Archive:  SDN Dialect Corpus v1.0.zip
   creating: SDN Dialect Corpus v1.0/
   creating: SDN Dialect Corpus v1.0/transcripts/
  inflating: SDN Dialect Corpus v1.0/transcripts/HM.csv  
  inflating: SDN Dialect Corpus v1.0/transcripts/WB.csv  
   creating: SDN Dialect Corpus v1.0/wav/
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0001.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0002.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0003.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0004.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0005.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0006.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0007.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0008.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0009.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0010.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0011.wav  
  inflating: SDN Dialect Corpus v1.0/wav/hm_01_0012.wav  
  inflating: SDN Dia

## Preparing the dataset  ##

First merging HM.csv and WB.csv files into one file sdn.csv, 
Second split the data to train and validation sets

In [ ]:
import pandas as pd
from glob import glob

interesting_files = glob("/content/SDN Dialect Corpus v1.0/transcripts/*.csv") # it grabs all the csv files from the directory you mention here

df_list = []
for filename in sorted(interesting_files):

    df_list.append(pd.read_csv(filename))
    full_df = pd.concat(df_list, sort=True)

# save the final file in same/different directory:
    full_df.to_csv("/content/SDN Dialect Corpus v1.0/transcripts/sdn.csv", index=False)

In [ ]:
# -todo- apply Buckwalter or CODA (Conventional Orthography for Dialectal Arabic) transliteration for te dataset
from lang_trans.arabic import buckwalter
# converting buckwalter format to arabic letters
# text = buckwalter.untransliterate(text)
# text = re.sub("\s+", " ", text) # remove multiple spaces
sdn_full_dataset = pd.read_csv('/content/SDN Dialect Corpus v1.0/transcripts/sdn.csv')
sdn_full_dataset['text'] = sdn_full_dataset['text'].apply(lambda t: buckwalter.transliterate(t))
sdn_full_dataset['filename'] = sdn_full_dataset['filename'].apply(lambda f:'/content/SDN Dialect Corpus v1.0/wav/{}.wav'.format(f))
sdn_full_dataset

,filename,text
0,/content/SDN Dialect Corpus v1.0/wav/hm_01_000...,tEbAn tEbAn xAlS tEbAn
1,/content/SDN Dialect Corpus v1.0/wav/hm_01_000...,tEbAn tEbAn xAlS tEbAn
2,/content/SDN Dialect Corpus v1.0/wav/hm_01_000...,nHwlk AlEnAyp Almkvfp
3,/content/SDN Dialect Corpus v1.0/wav/hm_01_000...,Al<sm Alkrym
4,/content/SDN Dialect Corpus v1.0/wav/hm_01_000...,mHswbk AlnEysAn Alfy Al$Er HsAn
...,...,...
3544,/content/SDN Dialect Corpus v1.0/wav/wb_03_024...,>wwwww >hlA >hlA >hlA yA wd AlHsyn
3545,/content/SDN Dialect Corpus v1.0/wav/wb_03_024...,<zykn
3546,/content/SDN Dialect Corpus v1.0/wav/wb_03_024...,mbswT kdy mbswT kdy mAlk yA wd AlHsyn
3547,/content/SDN Dialect Corpus v1.0/wav/wb_03_024...,>nA mA jbtA lkm Allylp xbr


In [ ]:
# from pathlib import Path
# import librosa
# import IPython.display as ipd
# SAMPLING_RATE = 16000
# # for i in range(len(sdn_full_dataset)):
# #   sdn_full_dataset.iloc[i]['audio'] = librosa.load(sdn_full_dataset.iloc[i]['filename'], sr=SAMPLING_RATE)

# # def audio_resampling(tr_dv_dst):
# audio = []
# trdst = sdn_full_dataset
# for i in range(len(sdn_full_dataset.index)):
#   # wav = librosa.load(sdn_full_dataset.iloc[i]['filename'], sr=SAMPLING_RATE)
#   audio.append(librosa.load(trdst.iloc[i]['filename'])[0])
#   # trdst.iloc[i]['audio'] = wav[0]
#   # trdst.iloc[i]['audio_sr'] = wav[1]
# # print(sdn_full_dataset['audio'])
# # wav
# len(audio)


In [ ]:
# # for i in range(len(audio)):
# trdst['audio'] = audio
#   # trdst['sr_audio'] = audio[i][1]
# # trdst
# # trdst['audio'] = trdst['audio'].apply(lambda f:"'{array({})'}'".format(f))
# type(audio[0])
# trdst

In [ ]:
import os
train_eval_data = sdn_full_dataset
if not os.path.isfile('./content/SDN Dialect Corpus v1.0/transcripts/train.csv'):
    eval_data = train_eval_data.sample(n=int(len(train_eval_data) * 0.1 ))
    train_data = train_eval_data[~train_eval_data.isin(eval_data)]
    train_data = train_data[train_data.filename.notnull()]
    
    train_data.to_csv('/content/SDN Dialect Corpus v1.0/transcripts/train.csv')
    eval_data.to_csv('/content/SDN Dialect Corpus v1.0/transcripts/eval.csv')
else:
    train_data = pd.read_csv('/content/SDN Dialect Corpus v1.0/transcripts/train.csv')
    eval_data = pd.read_csv('/content/SDN Dialect Corpus v1.0/transcripts/eval.csv')

In [ ]:
from datasets import load_dataset, load_metric


data_files = {
    "train": "/content/SDN Dialect Corpus v1.0/transcripts/train.csv", 
    "validation": "/content/SDN Dialect Corpus v1.0/transcripts/eval.csv",
}

dataset = load_dataset("csv", data_files=data_files)
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

print(train_dataset)
print(eval_dataset)

Using custom data configuration default-2dc3fe78eeefdd52


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-2dc3fe78eeefdd52/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

Dataset({
    features: ['Unnamed: 0', 'filename', 'text'],
    num_rows: 3195
})
Dataset({
    features: ['Unnamed: 0', 'filename', 'text'],
    num_rows: 354
})


In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(train_dataset)

,Unnamed: 0,filename,text
0,3422,/content/SDN Dialect Corpus v1.0/wav/wb_03_0120.wav,>kAn kdy bsm Allh AlrHmn AlrHym yAxwAnA Alnm$y Ely AlwlydAt w AlbnAt nklmn
1,2308,/content/SDN Dialect Corpus v1.0/wav/hm_25_0048.wav,m$ynA Albyt
2,3390,/content/SDN Dialect Corpus v1.0/wav/wb_03_0088.wav,Hqw E$An mA ynEs yqEd by Tryqp SHyHp yEny mA yxt mxdp w yAxd rAHtw kdA w bEd $wyp tlqAhw
3,1575,/content/SDN Dialect Corpus v1.0/wav/hm_17_0055.wav,>ywA
4,2576,/content/SDN Dialect Corpus v1.0/wav/hm_28_0070.wav,xlAS >nA bklm lyk nAs AlHlAl AlEly AlbHr jt Elyhm
5,2701,/content/SDN Dialect Corpus v1.0/wav/hm_29_0099.wav,Ebd AlEZym >Hmd Ebd AlqAdr
6,27,/content/SDN Dialect Corpus v1.0/wav/hm_01_0028.wav,wAllh yAxy AlfAtHp mEAk AlfAtHp yAxwy w>$try lynA mEAk mndlyn
7,712,/content/SDN Dialect Corpus v1.0/wav/hm_10_0037.wav,w myn qAl lyk >nA xAnqAny Ebrp w dAyr >fqEA
8,2499,/content/SDN Dialect Corpus v1.0/wav/hm_27_0075.wav,w Allh Twl bAlk Alnfws kAn <tTAybt AlEnqryb b$yl myp
9,2703,/content/SDN Dialect Corpus v1.0/wav/hm_29_0101.wav,syd >Hmd mHmd


In [ ]:
# This step may not be applicable to the transliterated dataset for Buckealker depends on upper cases and $ symbol

# import re
# chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

# def remove_special_characters(batch):
#     batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower() + " "
#     return batch

In [ ]:
# This step may not be applicable to the transliterated dataset for Buckealker depends on upper cases and $ symbol

# train_dataset = train_dataset.map(remove_special_characters)
# eval_dataset = eval_dataset.map(remove_special_characters)

In [ ]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [ ]:
vocab_train = train_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_dataset.column_names)
vocab_test = eval_dataset.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=eval_dataset.column_names)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_test["vocab"][0]))

In [ ]:
vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{' ': 7,
 '!': 17,
 '$': 22,
 '&': 15,
 "'": 8,
 '*': 23,
 '<': 39,
 '>': 20,
 'A': 6,
 'D': 12,
 'E': 31,
 'H': 18,
 'S': 19,
 'T': 1,
 'Y': 26,
 'Z': 28,
 'b': 37,
 'd': 13,
 'f': 11,
 'g': 25,
 'h': 27,
 'j': 38,
 'k': 2,
 'l': 32,
 'm': 24,
 'n': 9,
 'p': 16,
 'q': 3,
 'r': 35,
 's': 5,
 't': 0,
 'v': 36,
 'w': 29,
 'x': 10,
 'y': 14,
 'z': 4,
 '|': 30,
 '}': 34,
 '؟': 33,
 'پ': 21}

In [ ]:
# delete پ for its typo from wb_01_0100 text
# del vocab_dict['پ'] 
#del vocab_dict['؟']

# put + instead of space ' ' to make it visiable 
vocab_dict['+'] = vocab_dict[' '] 
del vocab_dict[' '] 

In [ ]:
# add a padding token that corresponds to CTC's "blank token".
# The "blank token" is a core component of the CTC algorithm

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
len(vocab_dict)
vocab_dict

{'!': 17,
 '$': 22,
 '&': 15,
 "'": 8,
 '*': 23,
 '+': 7,
 '<': 39,
 '>': 20,
 'A': 6,
 'D': 12,
 'E': 31,
 'H': 18,
 'S': 19,
 'T': 1,
 'Y': 26,
 'Z': 28,
 '[PAD]': 41,
 '[UNK]': 40,
 'b': 37,
 'd': 13,
 'f': 11,
 'g': 25,
 'h': 27,
 'j': 38,
 'k': 2,
 'l': 32,
 'm': 24,
 'n': 9,
 'p': 16,
 'q': 3,
 'r': 35,
 's': 5,
 't': 0,
 'v': 36,
 'w': 29,
 'x': 10,
 'y': 14,
 'z': 4,
 '|': 30,
 '}': 34,
 '؟': 33,
 'پ': 21}

In [ ]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

In [ ]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="+")

##Create XLSR-Wav2Vec2 Feature Extractor##

In [ ]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=True)

In [ ]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)
processor.save_pretrained("/content/drive/MyDrive/wav2vec2-large-xlsr-sdn-dialect")

##Pre-process Data (audio files)##

In [ ]:
# import seaborn as sb
# import librosa

# lens = []
# count = 0

# for _, row in sdn_full_dataset.iterrows():
#    wave, _ = librosa.load(row['filename'])
#    if wave is not None:
#        count += 1
#        lens.append(len(wave))
# secs = [x / 10000 for x in lens]       
# print(count)
# sb.distplot(secs, kde = False)

In [ ]:
train_dataset[100]
# train_dataset[0]["audio"][7:-8].replace('\n', '')

{'Unnamed: 0': 111,
 'filename': '/content/SDN Dialect Corpus v1.0/wav/hm_02_0081.wav',
 'text': '  wjdthw wjdthw wjdthw AlHmd llh >$krk yA Allh >$krk yA rb'}

In [ ]:
# import IPython.display as ipd
# import numpy as np
# import random

# rand_int = random.randint(0, len(train_dataset)-1)
# array = train_dataset[rand_int]["audio"]
# ipd.Audio(data=array, autoplay=True, rate=16000)

In [ ]:
import librosa
SAMPLING_RATE = 16000

def prepare_dataset(batch):
    audio = batch["filename"]

    # batched output is "un-batched"
    batch["input_values"] = processor(librosa.load(audio)[0], sampling_rate=SAMPLING_RATE).input_values[0]
    batch["input_length"] = len(batch["input_values"])

    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [ ]:
train_dataset = train_dataset.map(prepare_dataset, remove_columns=train_dataset.column_names, num_proc=4)
eval_dataset = eval_dataset.map(prepare_dataset, remove_columns=eval_dataset.column_names, num_proc=4)

#3:   0%|          | 0/798 [00:00<?, ?ex/s]

#0:   0%|          | 0/799 [00:00<?, ?ex/s]

#1:   0%|          | 0/799 [00:00<?, ?ex/s]

#2:   0%|          | 0/799 [00:00<?, ?ex/s]

#0:   0%|          | 0/89 [00:00<?, ?ex/s]

#1:   0%|          | 0/89 [00:00<?, ?ex/s]

#2:   0%|          | 0/88 [00:00<?, ?ex/s]

#3:   0%|          | 0/88 [00:00<?, ?ex/s]

In [ ]:
train_dataset[0]

{'input_length': 54744,
 'input_values': [0.007858357392251492,
  0.006611877121031284,
  0.002340378938242793,
  0.006789831910282373,
  0.007055589463561773,
  0.0013620754471048713,
  -0.0011076084338128567,
  0.005224546883255243,
  -7.938095222925767e-06,
  -0.004467378370463848,
  -0.0028749455232173204,
  -0.0019517363980412483,
  -0.005861196666955948,
  -0.0017966527957469225,
  -0.001664983108639717,
  0.0036631596740335226,
  0.009545021690428257,
  0.003209657035768032,
  -0.002870555268600583,
  -0.004397488199174404,
  -0.008715501055121422,
  -0.016281303018331528,
  -0.012405866757035255,
  -0.012196788564324379,
  -0.0071235052309930325,
  -0.00963680911809206,
  -0.012827306985855103,
  -0.006732574664056301,
  -0.0037713400088250637,
  -0.006617952138185501,
  -0.003216847777366638,
  -0.006714468821883202,
  -0.020578065887093544,
  -0.010725625790655613,
  -0.012039490975439548,
  -0.018650110810995102,
  -0.006876317318528891,
  -0.011075570248067379,
  -0.0195974

In [ ]:
max_input_length_in_sec = 5.0
train_dataset = train_dataset.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])
eval_dataset = eval_dataset.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, input_columns=["input_length"])

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

        # input_features, label_features = input_features.to(device), label_features.to(device)
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [ ]:
wer_metric = load_metric("wer")

In [ ]:
import numpy as np

def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [ ]:
from transformers import Wav2Vec2ForCTC

model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-large-xlsr-53", 
    attention_dropout=0.3,
    hidden_dropout=0.3,
    feat_proj_dropout=0.3,
    mask_time_prob=0.05,
    layerdrop=0.3,
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)
# jonatasgrosman/wav2vec2-large-xlsr-53-arabic

Downloading:   0%|          | 0.00/1.73k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-large-xlsr-53 were not used when initializing Wav2Vec2ForCTC: ['project_q.weight', 'quantizer.weight_proj.bias', 'project_q.bias', 'project_hid.bias', 'quantizer.codevectors', 'project_hid.weight', 'quantizer.weight_proj.weight']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to u

In [ ]:
model.freeze_feature_extractor()

/usr/local/lib/python3.7/dist-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1677: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  FutureWarning,


In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
# device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# model.to(device)
# data = data.to(device)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  # output_dir="/content/drive/MyDrive/wav2vec2-large-xlsr-sdn-dialect",
  output_dir="./wav2vec2-large-xlsr-sdn-dialect",
  group_by_length=True,
  per_device_train_batch_size=2,
  gradient_accumulation_steps=2,
  evaluation_strategy="steps",
  num_train_epochs=10,
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=3e-4,
  warmup_steps=50,
  save_total_limit=2,
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor,
)

Using amp half precision backend


In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1865
  Num Epochs = 10
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 2
  Total optimization steps = 4660


Step,Training Loss,Validation Loss,Wer
100,2.923900,2.982995,1.000000
200,2.967900,2.978680,1.000000
300,2.930100,2.980842,1.000000
400,2.891000,2.959640,1.000000
500,2.873600,2.873766,1.000000
600,2.851900,2.826822,1.000000
700,2.833400,2.811195,1.000000
800,2.945400,2.810792,1.000000
900,2.833100,2.814338,1.000000
1000,2.863600,2.806754,1.000000


The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 197
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-100
Configuration saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-100/config.json
Model weights saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-100/preprocessor_config.json
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 197
  Batch size = 8
Saving model checkpoint

Step,Training Loss,Validation Loss,Wer
100,2.923900,2.982995,1.000000
200,2.967900,2.978680,1.000000
300,2.930100,2.980842,1.000000
400,2.891000,2.959640,1.000000
500,2.873600,2.873766,1.000000
600,2.851900,2.826822,1.000000
700,2.833400,2.811195,1.000000
800,2.945400,2.810792,1.000000
900,2.833100,2.814338,1.000000
1000,2.863600,2.806754,1.000000


Saving model checkpoint to ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-1400
Configuration saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-1400/config.json
Model weights saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-1400/pytorch_model.bin
Feature extractor saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-1400/preprocessor_config.json
Deleting older checkpoint [wav2vec2-large-xlsr-sdn-dialect/checkpoint-1200] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `Wav2Vec2ForCTC.forward` and have been ignored: input_length. If input_length are not expected by `Wav2Vec2ForCTC.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 197
  Batch size = 8
Saving model checkpoint to ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-1500
Configuration saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoint-1500/config.json
Model weights saved in ./wav2vec2-large-xlsr-sdn-dialect/checkpoi

TrainOutput(global_step=4660, training_loss=3.0203314449654117, metrics={'train_runtime': 3255.5362, 'train_samples_per_second': 5.729, 'train_steps_per_second': 1.431, 'total_flos': 1.6195903324862874e+18, 'train_loss': 3.0203314449654117, 'epoch': 10.0})